In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("S08").getOrCreate()

24/11/11 09:27:58 WARN Utils: Your hostname, Shie-Hassaikai resolves to a loopback address: 127.0.1.1; using 10.140.226.15 instead (on interface wlp1s0)
24/11/11 09:27:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 09:27:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [43]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#Cambiar estructura
schemalocatel = StructType([
    StructField("d_codigo", StringType(), True),
    StructField("d_asenta", StringType(), True),
    StructField("d_tipo_asenta", StringType(), True),
    StructField("D_mnpio", StringType(), True),
    StructField("d_estado", StringType(), True),
    StructField("d_ciudad", StringType(), True),
    StructField("d_CP", StringType(), True),
    StructField("c_estado", StringType(), True),
    StructField("c_oficina", StringType(), True),
    StructField("c_CP", StringType(), True),
    StructField("c_tipo_asenta", StringType(), True),
    StructField("c_mnpio", StringType(), True),
    StructField("id_asenta_cpcons", StringType(), True),
    StructField("d_zona", StringType(), True),
    StructField("c_cve_ciudad", StringType(), True)
])
schemasepomex = StructType([
    StructField("id_folio", StringType(), True),
    StructField("fecha_solicitud", StringType(), True),
    StructField("hora_solicitud", StringType(), True),
    StructField("tipo_de_entrada", StringType(), True),
    StructField("tema_solicitud", StringType(), True),
    StructField("sexo", StringType(), True),
    StructField("edad", StringType(), True),
    StructField("coloniaSolicitud", StringType(), True),
    StructField("alcaldiaSolicitud", StringType(), True),
    StructField("CP_solicitud", StringType(), True),
    StructField("estatus", StringType(), True),
    StructField("fecha_concluido", StringType(), True),
    StructField("alcaldia_catalogo", StringType(), True),
    StructField("colonia_catalogo", StringType(), True),
    StructField("longitud", StringType(), True),
    StructField("latitud", StringType(), True)
])

sepomex_df = spark.read.option("header","true")\
    .option("delimiter",",")\
    .option("skipRows","1")\
    .option("encoding","UTF-8")\
    .schema(schemasepomex)\
    .csv("sepomex.csv")

sepomex_df\
    .select(['CP_solicitud','coloniaSolicitud','alcaldiaSolicitud','tema_solicitud','sexo'])\
    .distinct()\
    .createOrReplaceTempView("sepomex")

locatel = spark.read.option("header","true")\
    .option("delimiter",",")\
    .option("skipRows","1")\
    .option("encoding","UTF-8")\
    .schema(schemalocatel)\
    .csv("locatel2024.csv")

locatel\
    .select(['c_CP','D_mnpio','d_ciudad'])\
    .distinct()\
    .createOrReplaceTempView("locatel")

In [39]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  locatel|       true|
|         |  sepomex|       true|
+---------+---------+-----------+



In [44]:
strsql="""
    SELECT COALESCE(s.coloniaSolicitud, 'TOTAL_C') AS ciudad,
           COALESCE(s.tema_solicitud, 'TOTAL_TS') AS tema_solicitud,
           COALESCE(s.sexo, 'TOTAL_S') AS sexo,
           COUNT(1) AS nr,
           GROUPING_ID(s.coloniaSolicitud, s.tema_solicitud, s.sexo) AS grupo_id
    FROM sepomex s
    JOIN locatel l ON l.c_CP = s.CP_solicitud
    GROUP BY CUBE(s.coloniaSolicitud, s.tema_solicitud, s.sexo)
    ORDER BY 1, 2

"""

spark.sql(strsql).show(40, truncate=False)

24/11/11 10:36:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tema_solicitud, sexo, colonia_solicitud, alcaldia_solicitud, codigo_postal_solicitud
 Schema: tema_solicitud, sexo, coloniaSolicitud, alcaldiaSolicitud, CP_solicitud
Expected: coloniaSolicitud but found: colonia_solicitud
CSV file: file:///home/atenea/Macrodatos/sepomex.csv
24/11/11 10:36:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tipo_de_entrada, sexo, codigo_postal_solicitud
 Schema: D_mnpio, d_ciudad, c_CP
Expected: D_mnpio but found: tipo_de_entrada
CSV file: file:///home/atenea/Macrodatos/locatel2024.csv


+---------------+----------------------------------------+---------------+---+--------+
|ciudad         |tema_solicitud                          |sexo           |nr |grupo_id|
+---------------+----------------------------------------+---------------+---+--------+
|10 DE ABRIL    |FUGA DE AGUA                            |TOTAL_S        |5  |1       |
|10 DE ABRIL    |FUGA DE AGUA                            |NO ESPECIFICADO|5  |0       |
|10 DE ABRIL    |TOTAL_TS                                |TOTAL_S        |5  |3       |
|10 DE ABRIL    |TOTAL_TS                                |NO ESPECIFICADO|5  |2       |
|10 DE MAYO     |PODA / RETIRO ARBOL                     |TOTAL_S        |6  |1       |
|10 DE MAYO     |PODA / RETIRO ARBOL                     |MASCULINO      |6  |0       |
|10 DE MAYO     |TOTAL_TS                                |TOTAL_S        |6  |3       |
|10 DE MAYO     |TOTAL_TS                                |MASCULINO      |6  |2       |
|12 DE DICIEMBRE|ASESORÍA JURÍDI

In [33]:
strsql="""
    SELECT ciudad, tema_solicitud, nr
    FROM (
        SELECT 
            COALESCE(s.coloniaSolicitud, 'TOTAL_C') AS ciudad,
            COALESCE(l.d_ciudad, 'TOTAL_CIUDAD') AS tema_solicitud,  -- Usar d_ciudad en lugar de tema_solicitud
            COALESCE(l.D_mnpio, 'TOTAL_MNPIO') AS sexo,               -- Cambié l.c_mnpio por l.D_mnpio
            COUNT(*) AS nr,
            GROUPING_ID(s.coloniaSolicitud, l.d_ciudad, l.D_mnpio) AS grupo_id    -- Ajusté las columnas para que coincidan
        FROM locatel l
        JOIN sepomex s ON (l.c_CP = s.CP_solicitud)
        GROUP BY CUBE(s.coloniaSolicitud, l.d_ciudad, l.D_mnpio)                -- Usé d_ciudad en lugar de tema_solicitud
    ) x
    WHERE NOT (ciudad = 'TOTAL_C' AND tema_solicitud != 'TOTAL_CIUDAD')
    ORDER BY ciudad, grupo_id, tema_solicitud;

"""

spark.sql(strsql).show(40, truncate=False)

24/11/11 10:26:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tipo_de_entrada, sexo, codigo_postal_solicitud
 Schema: D_mnpio, d_ciudad, c_CP
Expected: D_mnpio but found: tipo_de_entrada
CSV file: file:///home/atenea/Macrodatos/locatel2024.csv
24/11/11 10:26:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tema_solicitud, sexo, colonia_solicitud, alcaldia_solicitud, codigo_postal_solicitud
 Schema: tema_solicitud, sexo, coloniaSolicitud, alcaldiaSolicitud, CP_solicitud
Expected: coloniaSolicitud but found: colonia_solicitud
CSV file: file:///home/atenea/Macrodatos/sepomex.csv


+---------------+---------------+---+
|ciudad         |tema_solicitud |nr |
+---------------+---------------+---+
|10 DE ABRIL    |MASCULINO      |1  |
|10 DE ABRIL    |MASCULINO      |1  |
|10 DE ABRIL    |NO ESPECIFICADO|1  |
|10 DE ABRIL    |NO ESPECIFICADO|1  |
|10 DE ABRIL    |NO ESPECIFICADO|1  |
|10 DE ABRIL    |MASCULINO      |2  |
|10 DE ABRIL    |NO ESPECIFICADO|3  |
|10 DE ABRIL    |TOTAL_CIUDAD   |1  |
|10 DE ABRIL    |TOTAL_CIUDAD   |1  |
|10 DE ABRIL    |TOTAL_CIUDAD   |2  |
|10 DE ABRIL    |TOTAL_CIUDAD   |1  |
|10 DE ABRIL    |TOTAL_CIUDAD   |5  |
|10 DE MAYO     |FEMENINO       |1  |
|10 DE MAYO     |MASCULINO      |1  |
|10 DE MAYO     |MASCULINO      |1  |
|10 DE MAYO     |NO ESPECIFICADO|1  |
|10 DE MAYO     |NO ESPECIFICADO|1  |
|10 DE MAYO     |NO ESPECIFICADO|1  |
|10 DE MAYO     |FEMENINO       |1  |
|10 DE MAYO     |MASCULINO      |2  |
|10 DE MAYO     |NO ESPECIFICADO|3  |
|10 DE MAYO     |TOTAL_CIUDAD   |1  |
|10 DE MAYO     |TOTAL_CIUDAD   |1  |
|10 DE MAYO 

In [48]:
strsql="""
    SELECT ciudad, tema_solicitud, nr
    FROM (
        SELECT 
            COALESCE(s.coloniaSolicitud, 'TOTAL_C') AS ciudad,
            COALESCE(s.tema_solicitud, 'TOTAL_TS') AS tema_solicitud,
            COALESCE(s.sexo, 'TOTAL_S') AS sexo,
            COUNT(*) AS nr,
            GROUPING_ID(s.coloniaSolicitud, s.tema_solicitud, s.sexo) AS grupo_id
        FROM locatel l
        JOIN sepomex s ON (l.c_CP = s.CP_solicitud)
        WHERE s.coloniaSolicitud IN ('Acapulco de Juarez', 'Acayucan', 'Actopan')
        GROUP BY CUBE(s.coloniaSolicitud, s.tema_solicitud, s.sexo)
    ) x
    WHERE NOT (ciudad = 'TOTAL_C' AND tema_solicitud != 'TOTAL_TS')
    AND NOT (ciudad = 'TOTAL_C' AND sexo != 'TOTAL_S')
    ORDER BY ciudad, grupo_id, tema_solicitud;

"""

spark.sql(strsql).show(4000, truncate=False)

24/11/11 10:46:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tipo_de_entrada, sexo, codigo_postal_solicitud
 Schema: D_mnpio, d_ciudad, c_CP
Expected: D_mnpio but found: tipo_de_entrada
CSV file: file:///home/atenea/Macrodatos/locatel2024.csv
24/11/11 10:46:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tema_solicitud, sexo, colonia_solicitud, alcaldia_solicitud, codigo_postal_solicitud
 Schema: tema_solicitud, sexo, coloniaSolicitud, alcaldiaSolicitud, CP_solicitud
Expected: coloniaSolicitud but found: colonia_solicitud
CSV file: file:///home/atenea/Macrodatos/sepomex.csv


+------+--------------+---+
|ciudad|tema_solicitud|nr |
+------+--------------+---+
+------+--------------+---+

